Implement Hidden Markov Model with categorical observations.
Use Expectation-Maximization to find the parameters.

The expectation step uses forward-backward algorithm

Notations follow Chapter 17, MLaPP, Kevin Murphy

Example: use historical `bullish to bearish ratio` obtained from American Association of Individual Investors website as an example data set


Formula used:
    
Forward:
$
\mathbf{\alpha}_t \propto \mathbf{\phi}_t \odot (A^T \mathbf{\alpha}_{t-1})  \hspace{500pt}
$

Backward:
$
\mathbf{\beta}_t = A (\mathbf{\phi}_{t+1} \odot \mathbf{\beta}_{t+1})   \hspace{500pt}
$

Forward-Backward:

1) $\gamma_t(j) \equiv P(z_t=j | x_{1:T})$  \hsapce{500pt}

$
\mathbf{\gamma}_t \propto \alpha_t \odot \beta_t   \hspace{500pt}
$
    
2) two-slice marginal

$
\xi_{t,t+1} \propto A \odot \big(\alpha_t ( \phi_{t+1} \odot \beta_{t+1})^T \big)   \hspace{500pt}
$


In [1]:
import numpy as np
import pandas as pd

In [2]:
class HMM(object):
    
    def __init__(self, obs, num_states):
        self.obs = obs
        self.K = num_states
        self.m = obs.max() + 1
        
    def normalize(self, p):
        norm = np.sum(p)
        p = p/norm
        return p, norm
    
    def forward(self, pi, A, B, obs):
        # pi: lenght-K, prior distribution over K states
        # A: KxK transition matrix, A[i, j] is the transition probability from state i to state j
        # B: mxK, each column is a state specific distribution over observations
        # obs: one given observation        
        K = A.shape[0]
        T = obs.size
        alpha = np.zeros((K, T))
        norms = np.zeros(T)
        
        alpha[:, 0] = B[obs[0], :] * pi
        alpha[:, 0], norms[0] = self.normalize(alpha[:, 0])
        for t in range(1, T):
            alpha[:, t] = B[obs[t], :] * (A.T.dot(alpha[:, t-1]))
            alpha[:, t], norms[t] = self.normalize(alpha[:, t])
        return alpha, norms
    
    def backward(self, pi, A, B, obs, alpha, norms):
        # pi: lenght-K, prior distribution over K states
        # A: KxK transition matrix, A[i, j] is the transition probability from state i to state j
        # B: mxK, each column is a state specific distribution over observations
        # obs: one given observation        
        # alpha: from the forward algorithm        
        K = A.shape[0]
        T = obs.size
        beta = np.zeros((K, T))
        beta[:, T-1] = 1
        beta[:, T-1] = beta[:, T-1] / norms[T-1]
        for t in range(T-2, -1, -1):
            beta[:, t] = A.dot(B[obs[t+1], : ] * beta[:, t+1])
            beta[:, t] = beta[:, t] / norms[t]
            # beta[:, t] is normalized with the same factor that normalized alpha[:, t], this
            #  ensures that gamma[:, t] is normalized
        return beta
    
    def forward_backward(self, A, B, obs, alpha, beta):
        # A: KxK transition matrix, A[i, j] is the transition probability from state i to state j
        # B: mxK, each column is a state specific distribution over observations
        # obs: one given observation
        # alpha: from the forward algorithm        
        # beta: from the backward algorithm
        K = A.shape[0]
        T = obs.size
        gamma = np.zeros((K, T))
        for t in range(T):
            gamma[:, t] = alpha[:, t] * beta[:, t]
            gamma[:, t], _ = self.normalize(gamma[:, t])
        
        Xi = np.zeros((K, K))
        for t in range(T-1):
            tmp = A * np.outer(alpha[:, t], B[obs[t+1], :] * beta[:, t+1])
            tmp = tmp/np.sum(tmp)
            Xi += tmp
        
        return gamma, Xi
    
    def expectation(self, pi, A, B, obs):
        alpha, norms = self.forward(pi, A, B, obs)
        beta = self.backward(pi, A, B, obs, alpha, norms)
        gamma, Xi = self.forward_backward(A, B, obs, alpha, beta)
        return alpha, norms, beta, gamma, Xi
    
    def maximization(self, gamma, Xi, obs):
        K = self.K
        m = self.m
        T = obs.size
        
        pi = gamma[:, 0].copy()
        
        A = Xi.copy()
        for i in range(K):
            A[i, :], _ = self.normalize(A[i, :])
            
        B = np.zeros((m, K))
        for l in range(m):
            for t in range(T):
                if obs[t] == l:
                    B[l, :] += gamma[:, t]
        for j in range(K):
            B[:, j], _ = self.normalize(B[:, j])
        
        return pi, A, B
    
    def train_EM(self, n_iters=1000):
        
        printing_frequency = n_iters // 10
        
        # initialization
        K = self.K
        m = self.m
        obs = self.obs
        T = obs.size
        
        # Initialization
        pi = np.random.random(K)
        pi, _ = self.normalize(pi)
        
        A = np.random.random((K, K))
        for j in range(K):
            A[j, :], _ = self.normalize(A[j, :])

        B = np.random.random((m, K))
        for j in range(K):
            B[:, j], _ = self.normalize(B[:, j])
            
        # EM iterations    
        for counter in range(n_iters):            
            alpha, norms, beta, gamma, Xi = self.expectation(pi, A, B, obs)
            pi, A, B = self.maximization(gamma, Xi, obs)
            if counter % printing_frequency == 0:
                cost_function = -np.sum(np.log(norms))
                print('iteration: {0:}, cost function: {1:}'.format(counter, cost_function))
        return pi, A, B
            

In [3]:
def viterbi(obs, pi, A, B):
    #
    # assume that there are K hidden states
    #   each state has a distribution over M possible outputs, which we observe
    #
    # Inputs:
    #     obs: observations from a sequence of length T
    #
    #     pi: K, prior distribution over states    
    #
    #     A: transition matrix for the hidden state, KxK
    #
    #     B : MxK, state specific distribution over possible outputs
    #       #
    # Internal variables:
    #     delta: KxT
    #         delta[j, t]: the most probable probability for state j at time t
    #
    #     a: KxT
    #        a[j, t]: the most probable state at time (t-1) that would lead to state j at time t
    
    T = obs.size
    K = A.shape[0]
    
    delta = np.zeros((K, T))
    a = np.zeros((K, T))
    
    delta[:, 0] = pi * B[obs[0], :]
    for t in range(1, T):
        for j in range(K):
            delta[j, t] = np.max(delta[:, t-1] * A[:, j] * B[obs[t], j])
            a[j, t] = np.argmax(delta[:, t-1] * A[:, j] * B[obs[t], j])
            
    # traceback 
    path = np.zeros(T)
    path[T-1] = np.argmax(delta[:, T-1])
    for t in range(T-2, -1, -1):
        path[t] = a[int(path[t+1]), t+1]
        
    return path

In [4]:
# http://www.aaii.com/sentimentsurvey/sent_results
data = pd.read_csv('AAII_raw.csv', header=0)

data['ratio'] = data.eval('Bullish / Bearish')
obs_max = data['ratio'].max()
obs_min = data['ratio'].min()
obs_delta = obs_max - obs_min

data['scaled_obs'] = data['ratio'].apply(lambda x: (x - obs_min)/obs_delta)
buckets = np.linspace(0, 1, 6)
data['digitized_obs'] = np.digitize(data['scaled_obs'], buckets) - 1

In [5]:
data.head()

,Date,Bullish,Neutral,Bearish,ratio,scaled_obs,digitized_obs
0,7-24-87,0.4,0.5,0.1,4.0,0.490909,2
1,7-31-87,0.3,0.5,0.3,1.0,0.109091,0
2,8-7-87,0.6,0.2,0.3,2.0,0.236364,1
3,8-14-87,0.5,0.4,0.2,2.5,0.300000,1
4,8-21-87,0.7,0.3,0.1,7.0,0.872727,4


In [6]:
obs = data['digitized_obs'].values

In [7]:
hmm = HMM(obs, 5)
pi, A, B = hmm.train_EM(1000)

iteration: 0, cost function: 3613.877366241577
iteration: 100, cost function: 925.5114808356143
iteration: 200, cost function: 925.2406293772912
iteration: 300, cost function: 925.1426442976081
iteration: 400, cost function: 924.8978398098704
iteration: 500, cost function: 920.9800093358799
iteration: 600, cost function: 919.0483644111841
iteration: 700, cost function: 918.4930880828682
iteration: 800, cost function: 914.3994293432136
iteration: 900, cost function: 913.4424556023216


In [8]:
pi

array([ 0.,  0.,  0.,  0.,  1.])

In [9]:
for i in range(A.shape[0]):
    row = ''
    for e in A[i, :]:
        e_str =  str(round(e, 4))
        row += e_str + '   '
    print(row)
    

0.9201   0.0   0.0785   0.0014   0.0   
0.0   0.6494   0.3506   0.0001   0.0   
0.1329   0.0   0.7484   0.1187   0.0   
0.0137   0.1544   0.0005   0.6867   0.1447   
0.0   0.0   0.0   0.3359   0.6641   


In [10]:
for i in range(B.shape[0]):
    row = ''
    for e in B[i, :]:
        e_str =  str(round(e, 4))
        row += e_str + '   '
    print(row)
    

1.0   0.9361   0.6727   0.2724   0.0674   
0.0   0.0015   0.3273   0.7276   0.5678   
0.0   0.0   0.0   0.0   0.1068   
0.0   0.0482   0.0   0.0   0.2065   
0.0   0.0142   0.0   0.0   0.0351   
0.0   0.0   0.0   0.0   0.0164   


In [11]:
most_probable_sequence = viterbi(obs, pi, A, B)
